<h1> Object Detection Exercises  <a align="center" target="_blank" href="https://colab.research.google.com/github/Marconi-Lab/dsa_2024/blob/main/dsa2024_yolov8_detection_exercises.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



Congratulations for completing the object detection walkthrough notebook!

Now that you are familiar with object detection and the YOLOv8 and Roboflow workflows, in this exercise notebook you will try out another zero-shot detection model called Grounding Dino. You will also annotate a few more images from the DSAIL-Porini Dataset and add it to the object detection dataset we used in the previous notebook. You will then use it to fine tune a YOLOv8 model and compare it with the previous runs.


## Exercise 1 : Dataset Annotation and Model Training

In this exercise you will annotate an additional 43 images to add on to the initial 365 annotated images. You will first download a dataset version with the original images (no augmentations) and upload them to a new project in your workspace. You will then upload the 43 unannotated images to the same project. After uploading the images, you will go to Roboflow and annotate the images. After annotating the images, you will create a new dataset version. You will then apply a few augmentations to the final dataset and load it into a colab session to train a YOLOv8 model variant of your choice

Read about the guide on the augmentations available on Roboflow (both the bounding box augmentations and image augmentations) and apply a few augmentations to the dataset. Download the dataset to Colab as we did in the previous notebook

### 1) Download the annotated dataset from Roboflow

In [1]:
## Install Roboflow
!pip install roboflow &>1

## Get roboflow api
from google.colab import userdata
roboflow_api = userdata.get('roboflow_api')

## Download the dataset
from roboflow import Roboflow
rf = Roboflow(api_key=roboflow_api)
project = rf.workspace("mltowardsobb").project("dsail-porini-detection-v2")
version = project.version(2) ## version 2 contains the original images without any augmentations
dataset = version.download("yolov8")

loading Roboflow workspace...
loading Roboflow project...
[WARNING] we noticed you are downloading a `yolov8` datasets but you don't have `ultralytics` installed. Roboflow `.deploy` supports only models trained with `ultralytics==8.0.196`, to intall it `pip install ultralytics==8.0.196`.



Extracting Dataset Version Zip to dsail-porini-detection-v2-2 in yolov8:: 100%|██████████| 942/942 [00:00<00:00, 8162.37it/s]


### 2) Upload the annotated dataset to a new Roboflow Project

In [5]:
PROJECT_NAME = 'dsail-porini-detection-fork'
DATASET_PATH = dataset.location
print(f"Dataset Path : {DATASET_PATH}")

Dataset Path : /content/dsail-porini-detection-v2-2


In [6]:
## Get your active Roboflow workspace
workspace = rf.workspace()
print(f"Active Workspace : {workspace.name}")

## Create a new project in the active workspace
project = workspace.create_project(
    project_name=PROJECT_NAME,
    project_type="object-detection",
    project_license="MIT",
    annotation='yolov8'
)

## Upload the dataset to the new project
workspace.upload_dataset(
    dataset_path=DATASET_PATH,
    num_workers=10,
    project_name=PROJECT_NAME,
    project_license="MIT",
    project_type="object-detection"
)

loading Roboflow workspace...
Active Workspace : MLtowardsobb
loading Roboflow project...
loading Roboflow project...
Uploading to existing project mltowardsobb/dsail-porini-detection-fork
[UPLOADED] /content/dsail-porini-detection-v2-2/test/images/BUSHBUCK_2021-11-02-17-29-49_jpg.rf.00a6f9286661bd317de7a31586fd7340.jpg (L5JpyJQJflDpaR68EmlJ) [1.4s] / annotations = OK [0.8s]
[UPLOADED] /content/dsail-porini-detection-v2-2/test/images/BUSHBUCK_2021-12-23-17-56-53_jpg.rf.061699955ea287a0e1405b28a9e2bba2.jpg (YsOype7vpuQEeaMCwe9J) [1.5s] / annotations = OK [0.7s]
[UPLOADED] /content/dsail-porini-detection-v2-2/test/images/IMPALA_2021-09-05-10-30-00_jpg.rf.59cce63814c2a1df432d068ac2e90a45.jpg (7IMmLlhI34xVBrQUGfLC) [1.5s] / annotations = OK [0.7s]
[UPLOADED] /content/dsail-porini-detection-v2-2/test/images/BUSHBUCK_2021-09-21-18-00-25_jpg.rf.74dc955f087b927724a4172ea5fc38cc.jpg (qpzxJOfrwhId3AHIEs38) [1.7s] / annotations = OK [1.0s]
[UPLOADED] /content/dsail-porini-detection-v2-2/test/imag

### 3) Download the images to annotate

In [7]:
## Download the images zip from github
!wget -q "https://raw.githubusercontent.com/Marconi-Lab/dsa_2024/main/dsail-porini-detection-unannotated-images.zip"

## Unzip
!unzip -q dsail-porini-detection-unannotated-images.zip

### 4) Upload the unannotated images to the same Roboflow Project

In [8]:
## Upload the unannotated images to same Roboflow Project
workspace.upload_dataset(
    dataset_path='/content/dsail-porini-detection-unannotated-images',
    num_workers=10,
    project_name=PROJECT_NAME,
    project_license="MIT",
    project_type="object-detection",
    batch_name='unannotated-new-images'
)

100%|██████████| 41/41 [00:00<00:00, 319640.27it/s]

loading Roboflow project...


Uploading to existing project mltowardsobb/dsail-porini-detection-fork
[UPLOADED] /content/dsail-porini-detection-unannotated-images/BACKGROUND_2021-11-03-10-01-20.jpg (qv3L9W6FMAr2wDoJIYqI) [1.5s]
[UPLOADED] /content/dsail-porini-detection-unannotated-images/BACKGROUND_2021-12-24-06-51-06.jpg (G3gd2XdgqOtfy9RV4ThJ) [1.7s]
[UPLOADED] /content/dsail-porini-detection-unannotated-images/BACKGROUND_2021-12-24-06-50-08.jpg (bNtOttphGuiigi6noYKz) [1.8s]
[UPLOADED] /content/dsail-porini-detection-unannotated-images/BUSHBUCK_2021-12-23-17-51-47.jpg (lkPDLWqSbLi7ZseNoEYP) [1.7s]
[UPLOADED] /content/dsail-porini-detection-unannotated-images/IMPALA_2021-09-18-10-49-32.jpg (gPNCH1okfORCf3dnJL3w) [1.7s]
[UPLOADED] /content/dsail-porini-detection-unannotated-images/BUSHBUCK_2021-12-23-17-56-17.jpg (pLL5f66p8E66Ko9KiHT3) [1.9s]
[UPLOADED] /content/dsail-porini-detection-unannotated-images/BACKGROUND_2021-11-03-10-00-52.jpg (LiZ8ze6EXWmPfodnAuwl) [2.0s]
[UPLOADED] /content/dsail-porini-detection-unann

### 5) Annotate the new batch of images on Roboflow

In [15]:
from IPython.display import Video
Video("https://raw.githubusercontent.com/Marconi-Lab/dsa_2024/main/assets/roboflow-screencast.webm")

 1) On Roboflow, copy and paste the class names as shown below on the project :

`bushbuck, impala, monkey, warthog, waterbuck, zebra`

 2) After updating the class names, go into the `Annotate` window and assign the images to yourself.

 3) Start annotating the images adhering to the bounding box annotation best practices. This [blog](https://blog.roboflow.com/tips-for-how-to-label-images/) by Roboflow highlights some of the best practices for bounding box annotations.

 4) While annotating the images, note that the filename of each image contains the name of the animal present in the image. If you can't tell the species of the animal in the image, this may guide you. Make sure to label background images which do not have any animals as `null`

### 6) Download the final dataset to Colab

1) After annotating all the 41 new images, add them to the dataset.    

2) Generate a dataset version with your choice of augmentations (both image level and bounding box level augmentations). You can read about Roboflow image augmentations [here](https://docs.roboflow.com/datasets/image-augmentation). Be careful to avoid augmentations that may not apply to the dataset such as vertical flip since it is not something that is likely to occur in nature and there is not likely to be a vertically flipped animal in the test set.

3) After the dataset has been created, run the cell below to download the dataset version. Make sure to specify the version number you want to download

In [10]:
PROJECT_VERSION_NUMBER = 1 ## Specify the version number you want to load
version = project.version(PROJECT_VERSION_NUMBER)
dataset = version.download("yolov8")

Generating version still in progress. Progress: 98.6%
[WARNING] we noticed you are downloading a `yolov8` datasets but you don't have `ultralytics` installed. Roboflow `.deploy` supports only models trained with `ultralytics==8.0.196`, to intall it `pip install ultralytics==8.0.196`.
Exporting format yolov8 in progress : 85.0%
Version export complete for yolov8 format



Extracting Dataset Version Zip to dsail-porini-detection-fork-3 in yolov8:: 100%|██████████| 2302/2302 [00:00<00:00, 7398.38it/s]


In [11]:
## Write yaml file with the correct configuration

%%writefile {dataset.location}/dataset.yaml
names:
 0 : bushbuck
 1 : impala
 2 : monkey
 3 : warthog
 4 : waterbuck
 5 : zebra

nc: 6
test: test/images
train: train/images
val: valid/images

Writing /content/dsail-porini-detection-fork-3/dataset.yaml


### 7) Train YOLOv8 Model

In [12]:
## Install ultralytics
!pip install ultralytics &>1

## Install wandb
!pip install wandb &>1

## Import packages
import wandb
import os
from ultralytics import YOLO

In [13]:
from google.colab import userdata
wandb_api = userdata.get('wandb_api')
wandb.login(key=wandb_api)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [14]:
## Initialize a wandb run
RUN_NAME = 'updated-dataset-run' ## Specify run name
DATASET_PATH = dataset.location

## Initialize wandb run
run = wandb.init(project='dsail-porini-detection', name=RUN_NAME) ## Initialize the same wandb project we initialized in the previous notebook

## Save the README file that contains information about the augmentations done in roboflow
wandb.save(os.path.join(DATASET_PATH, 'README.roboflow.txt'))

wandb: Currently logged in as: vickruto. Use `wandb login --relogin` to force relogin


wandb: WARNING Saving files without folders. If you want to preserve subdirectories pass base_path to wandb.save, i.e. wandb.save("/mnt/folder/file.h5", base_path="/mnt")


['/content/wandb/run-20240516_102416-xd2i1vll/files/dsail-porini-detection-fork-3/README.roboflow.txt']

In [ ]:
## Load one of the model variants
model = YOLO('yolov8s.pt') ## Try other sizes ie 'yolov8[n,s,m,l,x].pt

## Train the model
NUM_EPOCHS = 50 ## Specify number of epochs to run
model.train(data=os.path.join(DATASET_PATH, 'dataset.yaml'),
            epochs=NUM_EPOCHS,
            name = RUN_NAME,
            save_dir=f'runs/detect/{RUN_NAME}')

100%|██████████| 21.5M/21.5M [00:00<00:00, 337MB/s]


Ultralytics YOLOv8.2.16 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/content/dsail-porini-detection-fork-3/dataset.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=updated-dataset-run, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, 

100%|██████████| 755k/755k [00:00<00:00, 23.2MB/s]


Overriding model.yaml nc=80 with nc=6

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytics

100%|██████████| 6.23M/6.23M [00:00<00:00, 132MB/s]


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       6/50      4.16G     0.7561      1.157      1.092         11        640: 100%|██████████| 65/65 [00:27<00:00,  2.37it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:01<00:00,  1.75it/s]

                   all         70         95      0.616      0.652      0.661      0.412



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       7/50      4.15G     0.7837      1.115      1.114         29        640:  52%|█████▏    | 34/65 [00:12<00:09,  3.28it/s]

While the model is training, log into the wandb platform and compare the runs. Check if your model has lower loss levels towards the end of the training since you used more data

## Exercise 2 : Grounding Dino Zero-Shot Detection

In this exercise you will try out [`Grounding Dino`](https://github.com/IDEA-Research/GroundingDINO) another state of the art zero-shot detection model, and compare its performance to YOLO-World. This tutorial blog by Roboflow introduces Grounding Dino and explains in further detail how zero-shot detectors work. The tutorial also has an accompanying [Youtube Video](https://www.youtube.com/watch?v=cMa77r3YrDk) and [Colab Notebook](https://colab.research.google.com/github/roboflow-ai/notebooks/blob/main/notebooks/zero-shot-object-detection-with-grounding-dino.ipynb?ref=blog.roboflow.com#scrollTo=QE9aSXo9e1jR) that you can use as guides. Set up the model and run it on a few of the DSAIL-Porini images. Try out different descriptive prompts and check how the prompts affect the bounding box prediction. Also compare the inference speed to that of YOLO-World
